In [1]:
import simpy


def resource_user(env, resource):
    request = resource.request()  # Generate a request event
    yield request  # Wait for access
    yield env.timeout(1)  # Do something
    resource.release(request)  # Release the resource


env = simpy.Environment()
res = simpy.Resource(env, capacity=1)
user = env.process(resource_user(env, res))
env.run()


In [2]:
def resource_user(env, resource):
    with resource.request() as req:  # Generate a request event
        yield req  # Wait for access
        yield env.timeout(1)  # Do something
        # Resource released automatically


user = env.process(resource_user(env, res))
env.run()


In [3]:
res = simpy.Resource(env, capacity=1)


def print_stats(res):
    print(f"{res.count} of {res.capacity} slots are allocated.")
    print(f"  Users: {res.users}")
    print(f"  Queued events: {res.queue}")


def user(res):
    print_stats(res)
    with res.request() as req:
        yield req
        print_stats(res)
    print_stats(res)


procs = [env.process(user(res)), env.process(user(res))]
env.run()


0 of 1 slots are allocated.
  Users: []
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x29c3c313f10>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x29c3c313f10>]
  Queued events: [<Request() object at 0x29c3c459340>]
0 of 1 slots are allocated.
  Users: []
  Queued events: [<Request() object at 0x29c3c459340>]
1 of 1 slots are allocated.
  Users: [<Request() object at 0x29c3c459340>]
  Queued events: []
0 of 1 slots are allocated.
  Users: []
  Queued events: []


In [4]:
def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print(f"{name} requesting at {env.now} with priority={prio}")
        yield req
        print(f"{name} got resource at {env.now}")
        yield env.timeout(3)


env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()


1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
3 got resource at 3
2 got resource at 6


In [5]:
def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print(f"{name} requesting at {env.now} with priority={prio}")
        yield req
        print(f"{name} got resource at {env.now}")
        try:
            yield env.timeout(3)
        except simpy.Interrupt as interrupt:
            by = interrupt.cause.by
            usage = env.now - interrupt.cause.usage_since
            print(f"{name} got preempted by {by} at {env.now}" f" after {usage}")


env = simpy.Environment()
res = simpy.PreemptiveResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()


1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
1 got preempted by <Process(resource_user) object at 0x29c3c459e20> at 2 after 2
3 got resource at 2
2 got resource at 5


In [6]:
def user(name, env, res, prio, preempt):
    with res.request(priority=prio, preempt=preempt) as req:
        try:
            print(f"{name} requesting at {env.now}")
            assert isinstance(env.now, int), type(env.now)
            yield req
            assert isinstance(env.now, int), type(env.now)
            print(f"{name} got resource at {env.now}")
            yield env.timeout(3)
        except simpy.Interrupt:
            print(f"{name} got preempted at {env.now}")


env = simpy.Environment()
res = simpy.PreemptiveResource(env, capacity=1)
A = env.process(user("A", env, res, prio=0, preempt=True))
env.run(until=1)  # Give A a head start


A requesting at 0
A got resource at 0


In [7]:
B = env.process(user("B", env, res, prio=-2, preempt=False))
C = env.process(user("C", env, res, prio=-1, preempt=True))
env.run()


B requesting at 1
C requesting at 1
B got resource at 3
C got resource at 6


In [8]:
class GasStation:
    def __init__(self, env):
        self.fuel_dispensers = simpy.Resource(env, capacity=2)
        self.gas_tank = simpy.Container(env, init=100, capacity=1000)
        self.mon_proc = env.process(self.monitor_tank(env))

    def monitor_tank(self, env):
        while True:
            if self.gas_tank.level < 100:
                print(f"Calling tanker at {env.now}")
                env.process(tanker(env, self))
            yield env.timeout(15)


def tanker(env, gas_station):
    yield env.timeout(10)  # Need 10 Minutes to arrive
    print(f"Tanker arriving at {env.now}")
    amount = gas_station.gas_tank.capacity - gas_station.gas_tank.level
    yield gas_station.gas_tank.put(amount)


def car(name, env, gas_station):
    print(f"Car {name} arriving at {env.now}")
    with gas_station.fuel_dispensers.request() as req:
        yield req
        print(f"Car {name} starts refueling at {env.now}")
        yield gas_station.gas_tank.get(40)
        yield env.timeout(5)
        print(f"Car {name} done refueling at {env.now}")


def car_generator(env, gas_station):
    for i in range(4):
        env.process(car(i, env, gas_station))
        yield env.timeout(5)


env = simpy.Environment()
gas_station = GasStation(env)
car_gen = env.process(car_generator(env, gas_station))
env.run(35)


Car 0 arriving at 0
Car 0 starts refueling at 0
Car 1 arriving at 5
Car 0 done refueling at 5
Car 1 starts refueling at 5
Car 2 arriving at 10
Car 1 done refueling at 10
Car 2 starts refueling at 10
Calling tanker at 15
Car 3 arriving at 15
Car 3 starts refueling at 15
Tanker arriving at 25
Car 2 done refueling at 30
Car 3 done refueling at 30


In [9]:
def producer(env, store):
    for i in range(100):
        yield env.timeout(2)
        yield store.put(f"spam {i}")
        print(f"Produced spam at", env.now)


def consumer(name, env, store):
    while True:
        yield env.timeout(1)
        print(name, "requesting spam at", env.now)
        item = yield store.get()
        print(name, "got", item, "at", env.now)


env = simpy.Environment()
store = simpy.Store(env, capacity=2)

prod = env.process(producer(env, store))
consumers = [env.process(consumer(i, env, store)) for i in range(2)]

env.run(until=5)


0 requesting spam at 1
1 requesting spam at 1
Produced spam at 2
0 got spam 0 at 2
0 requesting spam at 3
Produced spam at 4
1 got spam 1 at 4


In [10]:
from collections import namedtuple

Machine = namedtuple("Machine", "size, duration")
m1 = Machine(1, 2)  # Small and slow
m2 = Machine(2, 1)  # Big and fast

env = simpy.Environment()
machine_shop = simpy.FilterStore(env, capacity=2)
machine_shop.items = [m1, m2]  # Pre-populate the machine shop


def user(name, env, ms, size):
    machine = yield ms.get(lambda machine: machine.size == size)
    print(name, "got", machine, "at", env.now)
    yield env.timeout(machine.duration)
    yield ms.put(machine)
    print(name, "released", machine, "at", env.now)


users = [env.process(user(i, env, machine_shop, (i % 2) + 1)) for i in range(3)]
env.run()


0 got Machine(size=1, duration=2) at 0
1 got Machine(size=2, duration=1) at 0
1 released Machine(size=2, duration=1) at 1
0 released Machine(size=1, duration=2) at 2
2 got Machine(size=1, duration=2) at 2
2 released Machine(size=1, duration=2) at 4


In [11]:
env = simpy.Environment()
issues = simpy.PriorityStore(env)


def inspector(env, issues):
    for issue in [
        simpy.PriorityItem("P2", "#0000"),
        simpy.PriorityItem("P0", "#0001"),
        simpy.PriorityItem("P3", "#0002"),
        simpy.PriorityItem("P1", "#0003"),
    ]:
        yield env.timeout(1)
        print(env.now, "log", issue)
        yield issues.put(issue)


def maintainer(env, issues):
    while True:
        yield env.timeout(3)
        issue = yield issues.get()
        print(env.now, "repair", issue)


_ = env.process(inspector(env, issues))
_ = env.process(maintainer(env, issues))
env.run()


1 log PriorityItem(priority='P2', item='#0000')
2 log PriorityItem(priority='P0', item='#0001')
3 log PriorityItem(priority='P3', item='#0002')
3 repair PriorityItem(priority='P0', item='#0001')
4 log PriorityItem(priority='P1', item='#0003')
6 repair PriorityItem(priority='P1', item='#0003')
9 repair PriorityItem(priority='P2', item='#0000')
12 repair PriorityItem(priority='P3', item='#0002')
